In [3]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import seaborn as sns

import pandas as pd
import numpy as np
import datetime

In [4]:
from models import Candle, MarketTrade
from database import Database, MyWoWDatabase
import services.coinbase_services as cb
from coinbase.rest import RESTClient

In [5]:
client = cb.get_client()
db = Database('mywow.db')
dbms = MyWoWDatabase('mywow.db')

trading_pair = 'SWELL-USD'
start = datetime.datetime(year=2025, month=2, day=10, hour=1, minute=30, second=0, tzinfo=cb.LOCAL_TZ)
end = datetime.datetime(year=2025, month=2, day=10, hour=3, minute=00, second=0, tzinfo=cb.LOCAL_TZ)

In [6]:
# Assuming at root of project directory: "C:\...\MyWoW\"
import os
data_dir_path = os.path.join(os.getcwd(), 'data')
if not os.path.exists(data_dir_path):
    os.mkdir(data_dir_path)

analysis_dir_path = os.path.join(data_dir_path, 'analysis')
if not os.path.exists(analysis_dir_path):
    os.mkdir(analysis_dir_path)

DATA_FETCHED = False
history_header = ','.join(['trading_pair', 'start_date', 'end_date', 'created_date', 'last_updated']) + '\n'
analysis_history_file = os.path.join(analysis_dir_path, 'history.csv')
if not os.path.exists(analysis_history_file):
    with open(analysis_history_file, 'x') as f:
        f.write(history_header)
        f.write(','.join([trading_pair, start.isoformat(), end.isoformat(), datetime.datetime.now(tz=cb.LOCAL_TZ).isoformat(), datetime.datetime.now(tz=cb.LOCAL_TZ).isoformat()]) + '\n')
else:
    with open(analysis_history_file, 'r') as f:
        header = f.readline()
        for line in f.readlines():
            prev_trading_pair, prev_start_date, prev_end_date, _, _ = line.strip('\n').split(',')
            if prev_trading_pair == trading_pair and prev_start_date == start.isoformat() and prev_end_date == end.isoformat():
                DATA_FETCHED = True

In [7]:
def fetch_and_upload_data(client: RESTClient, trading_pair: str, start_time: datetime.datetime, end_time: datetime.datetime):
    # fetch all market trades between time range and upload to db
    market_trades = cb.fetch_market_trades(client, trading_pair, start_time, end_time, cb.CANDLES_LIMIT_MAX)
    for market_trade_data in market_trades:
        market_trade = MarketTrade(market_trade_data)
        dbms.add_item(table_name='market_trade', values=market_trade.get_values())
    
    # fetch candles between same time range as market trades and upload to db
    candles = cb.fetch_market_trade_candles(client, trading_pair, start_time, end_time, cb.CANDLES_LIMIT_MAX)
    for candle_data in candles:
        candle = Candle(candle_data)
        dbms.add_item(table_name='market_candles', values=candle.get_values(market_trade_candle=True))

def reset_fetch_data(trading_pair: str, start_time: datetime.datetime, end_time: datetime.datetime):
    res = dbms.get_items(
        table_name='market_trade', 
        where_statement=f"WHERE trading_pair='{trading_pair}' AND time BETWEEN '{start_time.isoformat()}' AND '{end_time.isoformat()}'"
        )
    market_trades = [MarketTrade(data) for data in res] 
    for market_trade in market_trades:
        dbms.remove_item(table_name='market_trade', values={'trade_id':market_trade.trade_id})

    res = dbms.get_items(
        table_name='market_candles',
        where_statement=f"WHERE trading_pair='{trading_pair}' AND time BETWEEN '{start_time.isoformat()}' AND '{end_time.isoformat()}'"
    )
    candles = [Candle(data) for data in res]
    for candle in candles:
        dbms.remove_item(table_name='market_candles', values={'candle_id':candle.candle_id})

In [8]:
# reset_fetch_data(trading_pair=trading_pair, start_time=start, end_time=end)
if not DATA_FETCHED:
    fetch_and_upload_data(trading_pair=trading_pair, start_time=start, end_time=end)

In [9]:
res = dbms.get_items( 
    table_name='market_candles', 
    where_statement=f"WHERE trading_pair='{trading_pair}' AND time BETWEEN '{start.isoformat()}' AND '{end.isoformat()}'")
market_candles = [Candle(candle) for candle in res]

df_candles = pd.DataFrame(data=[candle.to_dict() for candle in market_candles])
df_candles['timestamp'] = df_candles['date'].transform(lambda x: x.timestamp())
df_candles['time'] = df_candles['date'].transform(lambda x: x.strftime('%H:%M:%S'))
df_candles['hour'] = df_candles['date'].transform(lambda x: x.hour)
df_candles['minute'] = df_candles['date'].transform(lambda x: x.minute)
df_candles.sort_values(by='date', inplace=True)
df_candles.dtypes
df_candles.head()

,candle_id,date,start,trading_pair,open,high,low,close,volume,timestamp,time,hour,minute
66,SWELL-1739179800,2025-02-10 01:30:00-08:00,1739179800,SWELL-USD,0.01171,0.01172,0.01168,0.01172,12935.0,1.739180e+09,01:30:00,1,30
65,SWELL-1739179980,2025-02-10 01:33:00-08:00,1739179980,SWELL-USD,0.01173,0.01178,0.01172,0.01176,94445.0,1.739180e+09,01:33:00,1,33
64,SWELL-1739180220,2025-02-10 01:37:00-08:00,1739180220,SWELL-USD,0.01166,0.01166,0.01164,0.01164,149042.0,1.739180e+09,01:37:00,1,37
63,SWELL-1739180760,2025-02-10 01:46:00-08:00,1739180760,SWELL-USD,0.01173,0.01186,0.01173,0.01180,94002.0,1.739181e+09,01:46:00,1,46
62,SWELL-1739181060,2025-02-10 01:51:00-08:00,1739181060,SWELL-USD,0.01176,0.01176,0.01173,0.01176,43982.0,1.739181e+09,01:51:00,1,51


In [10]:
res = dbms.get_items(
    table_name='market_trade',
    where_statement=f"WHERE trading_pair='{trading_pair}' AND time >= '{start.isoformat()}'")
market_trades = [MarketTrade(trade) for trade in res]
market_trade_data = []
for market_trade in market_trades:
    data = market_trade.to_dict()
    data['total'] = market_trade.total
    market_trade_data.append(data)

df_trades = pd.DataFrame(data=market_trade_data)
df_trades['date'] = df_trades['time']
df_trades['timestamp'] = df_trades['date'].transform(lambda x: x.timestamp())
df_trades['time'] = df_trades['date'].transform(lambda x: x.strftime('%H:%M:%S'))
df_trades['hour'] = df_trades['date'].transform(lambda x: x.hour)
df_trades['minute'] = df_trades['date'].transform(lambda x: x.minute)
df_trades['second'] = df_trades['date'].transform(lambda x: x.second)
df_trades.head()

,trade_id,trading_pair,price,size,time,side,bid,ask,exchange,total,date,timestamp,hour,minute,second
0,1135065,SWELL-USD,0.01194,6678.0,01:58:25,BUY,0.0,0.0,UKNOWN_EXCHANGE,79.73532,2025-02-10 01:58:25.908019-08:00,1.739182e+09,1,58,25
1,1135064,SWELL-USD,0.01200,9414.0,01:58:25,BUY,0.0,0.0,UKNOWN_EXCHANGE,112.96800,2025-02-10 01:58:25.737970-08:00,1.739182e+09,1,58,25
2,1135062,SWELL-USD,0.01203,9414.0,01:58:25,BUY,0.0,0.0,UKNOWN_EXCHANGE,113.25042,2025-02-10 01:58:25.732422-08:00,1.739182e+09,1,58,25
3,1135063,SWELL-USD,0.01200,12552.0,01:58:25,BUY,0.0,0.0,UKNOWN_EXCHANGE,150.62400,2025-02-10 01:58:25.732422-08:00,1.739182e+09,1,58,25
4,1135060,SWELL-USD,0.01199,767.0,01:58:25,SELL,0.0,0.0,UKNOWN_EXCHANGE,-9.19633,2025-02-10 01:58:25.647777-08:00,1.739182e+09,1,58,25


In [11]:
fig_height = 550
fig_width = 850

In [12]:
fig_pricechange = go.Figure(
    data=[go.Candlestick(
        x=df_candles.date,
        open=df_candles.open,
        high=df_candles.high,
        low=df_candles.low,
        close=df_candles.close
)])
fig_pricechange.update_layout(
    title=f'Price Change for {trading_pair} on {start.strftime('%m/%d/%y')} between {start.strftime('%H:%M')} and {end.strftime('%H:%M')}',
    yaxis=dict(
        title=dict(text='Price')
    ),
    xaxis=dict(
        title=dict(text='Time'),
        dtick=5*60*1000.0
    ),
    height=fig_height,
    width=fig_width,
    xaxis_rangeslider_visible=False)
fig_pricechange.show()

In [13]:
df_trades['hm_time'] = df_trades['date'].transform(lambda x:datetime.time(hour=x.hour, minute=x.minute))
df_trade_counts = df_trades.groupby(['hm_time', 'side'], as_index=False)['trade_id'].count()
df_trade_counts.rename(columns={'trade_id':'counts'}, inplace=True)
df_trade_counts.head()

,hm_time,side,counts
0,01:30:00,BUY,1
1,01:30:00,SELL,1
2,01:32:00,SELL,2
3,01:33:00,SELL,4
4,01:36:00,SELL,4


In [14]:
fig_tradecounts = px.bar(df_trade_counts, x='hm_time', y='counts', color='side')
fig_tradecounts.update_layout(
    title=f'{trading_pair} Market Trade Counts on {start.strftime('%m/%d/%y')} between {start.strftime('%H:%M')} and {end.strftime('%H:%M')}',
    yaxis=dict(
        title=dict(text='Counts')
    ),
    xaxis=dict(
        title=dict(text='Time')
    ),
    height=fig_height,
    width=fig_width
)
fig_tradecounts.show()

In [15]:
df_trade_totals = df_trades.groupby(['hm_time', 'side'], as_index=False)['total'].sum()
df_trade_totals.head()

,hm_time,side,total
0,01:30:00,BUY,11.23616
1,01:30:00,SELL,-2.92750
2,01:32:00,SELL,-137.34356
3,01:33:00,SELL,-505.74156
4,01:36:00,SELL,-601.94368


In [16]:
fig_tradetotals = px.bar(
    df_trade_totals,
    x='hm_time',
    y='total',
    color='side'
)
fig_tradetotals.update_layout(
    title=f"{trading_pair} Market Trade Totals on {start.strftime('%b %d')} between {start.strftime('%H:%M')} and {end.strftime('%H:%M')}",
    xaxis=dict(
        title=dict(text='Time')
    ),
    yaxis=dict(
        title=dict(text='Total')
    ),
    height=fig_height,
    width=fig_width,
)
fig_tradetotals.show()

In [17]:
fig_pricechange.show()
fig_tradecounts.show()
fig_tradetotals.show()

In [18]:
# Weekday Analysis
trading_pair = 'ARB-USD'
weekday_start = datetime.datetime(year=2024, month=8, day=1, tzinfo=cb.LOCAL_TZ)
weekday_end = datetime.datetime(year=2025, month=2, day=1, tzinfo=cb.LOCAL_TZ) - datetime.timedelta(seconds=1)
print(trading_pair)
print(weekday_start)
print(weekday_end)

ARB-USD
2024-08-01 00:00:00-08:00
2025-01-31 23:59:59-08:00


In [19]:
# candles = cb.get_asset_candles(client=client, product_id=trading_pair, granularity=cb.Granularity.ONE_DAY, start=weekday_start, end=weekday_end)
# for candle_data in candles:
#     candle = Candle(candle_data)
#     dbms.add_item(table_name='candles', values=candle.get_values())
# print(f'Fetched and uploaded {len(candles)} candles')

In [20]:
res = dbms.get_items(table_name='candles', where_statement=f"WHERE trading_pair='{trading_pair}' AND date BETWEEN '{weekday_start.date()}' AND '{weekday_end.date()}'")
weekday_candles = [Candle(data) for data in res]

df_weekday_candles = pd.DataFrame(data=[candle.to_dict() for candle in weekday_candles])

df_weekday_candles['price_change'] = df_weekday_candles['close'] - df_weekday_candles['open']
df_weekday_candles['price_direction'] = df_weekday_candles['price_change'].transform(lambda x: 'Positive' if x > 0 else 'Negative')
df_weekday_candles['weekday'] = df_weekday_candles['date'].transform(lambda x: x.strftime('%A'))
df_weekday_candles['month'] = df_weekday_candles['date'].transform(lambda x: x.month)
df_weekday_candles['month-year'] = df_weekday_candles['date'].transform(lambda x: x.strftime('%Y-%m'))
df_weekday_candles['percent_change'] = (df_weekday_candles['price_change'] / df_weekday_candles['open']) * 100

df_weekday_candles.head()

,candle_id,date,start,trading_pair,open,high,low,close,volume,price_change,price_direction,weekday,month,month-year,percent_change
0,ARB-1734480000,2024-12-17 16:00:00-08:00,1734480000,ARB-USD,0.9492,0.9558,0.8510,0.8615,9782590.86,-0.0877,Negative,Tuesday,12,2024-12,-9.239359
1,ARB-1734393600,2024-12-16 16:00:00-08:00,1734393600,ARB-USD,1.0034,1.0167,0.9343,0.9489,6079050.40,-0.0545,Negative,Monday,12,2024-12,-5.431533
2,ARB-1734307200,2024-12-15 16:00:00-08:00,1734307200,ARB-USD,1.0169,1.0546,0.9643,1.0032,11107753.24,-0.0137,Negative,Sunday,12,2024-12,-1.347232
3,ARB-1735516800,2024-12-29 16:00:00-08:00,1735516800,ARB-USD,0.7414,0.7702,0.7161,0.7319,14215223.65,-0.0095,Negative,Sunday,12,2024-12,-1.281360
4,ARB-1735430400,2024-12-28 16:00:00-08:00,1735430400,ARB-USD,0.7755,0.7819,0.7326,0.7411,2435685.80,-0.0344,Negative,Saturday,12,2024-12,-4.435848


In [21]:
months = list(map(int, df_weekday_candles['month'].unique()))
month_years = list(df_weekday_candles['month-year'].unique())
month_years_order = sorted(month_years)
weekdays_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

In [22]:
wkday_price_direction_counts = df_weekday_candles.groupby(['month-year', 'weekday', 'price_direction'], as_index=False)['candle_id'].count()
wkday_price_direction_counts.rename(columns={'candle_id':'counts'}, inplace=True)
wkday_pdc_fig = px.bar(wkday_price_direction_counts, x='weekday', y='counts', color='price_direction', 
             barmode='group', facet_row='month-year', category_orders={'month':month_years_order, 'weekday':weekdays_order})
wkday_pdc_fig.update_layout(
    height=fig_height,
    width=fig_width
)
wkday_pdc_fig.for_each_annotation(lambda x: x.update(text=x.text.split('=')[-1]))
wkday_pdc_fig.show()

In [23]:
wkday_price_change_total = df_weekday_candles.groupby(['month-year', 'weekday', 'price_direction'], as_index=False)['percent_change'].sum()
wkday_price_change_total.rename(columns={'percent_change':'% diff'}, inplace=True)
fig_wkday_pct = px.bar(wkday_price_change_total, x='weekday', y='% diff', color='price_direction',
                       barmode='group', facet_row='month-year', category_orders={'month-year':month_years_order, 'weekday':weekdays_order})
fig_wkday_pct.update_layout(
    height=fig_height+150,
    width=fig_width
)
fig_wkday_pct.for_each_annotation(lambda x: x.update(text=x.text.split('=')[-1]))
fig_wkday_pct.show()

In [24]:
month_year_filter = '2024-11'
# average price change
avg_price_change = df_weekday_candles[df_weekday_candles['month-year']==month_year_filter].groupby(['month-year', 'weekday'], as_index=False)['price_change'].mean()
avg_price_change.rename(columns={'price_change':'avg change'}, inplace=True)
avg_price_change['color'] = np.where(avg_price_change['avg change']<0, 'red', 'green')
# max price change
max_price_change = df_weekday_candles[df_weekday_candles['month-year']==month_year_filter].groupby(['month-year', 'weekday'], as_index=False)['price_change'].max()
max_price_change.rename(columns={'price_change':'max change'}, inplace=True)
max_price_change['color'] = np.where(max_price_change['max change']<0, 'red', 'green')
# min price change
min_price_change = df_weekday_candles[df_weekday_candles['month-year']==month_year_filter].groupby(['month-year', 'weekday'], as_index=False)['price_change'].min()
min_price_change.rename(columns={'price_change':'min change'}, inplace=True)
min_price_change['color'] = np.where(min_price_change['min change']<0, 'red', 'green')

avg_price_change.head()

,month-year,weekday,avg change,color
0,2024-11,Friday,0.043660,green
1,2024-11,Monday,-0.025750,red
2,2024-11,Saturday,-0.011600,red
3,2024-11,Sunday,0.035125,green
4,2024-11,Thursday,0.021100,green


In [ ]:
wkday_fig_height = 850
wkday_fig_width = 1050

# fig_apc = px.bar(avg_price_change, x='weekday', y='avg change', facet_row='month-year',
#                  barmode='group', category_orders={'weekday':weekdays_order})
# fig_apc.for_each_annotation(lambda x: x.update(text=x.text.split('=')[-1]))
fig_apc = go.Figure(data=go.Bar(
    x=avg_price_change['weekday'], y=avg_price_change['avg change'], marker_color=avg_price_change['color'],
))
fig_apc.update_layout(
    title_text=f"Average Price Change for {trading_pair} in {month_year_filter}",
    height=wkday_fig_height,
    width=wkday_fig_width,
    xaxis=dict(categoryorder='array', categoryarray=weekdays_order),
)
fig_apc.show()

In [26]:
# fig_maxpc = px.bar(max_price_change, x='weekday', y='max change', facet_row='month-year',
#                  barmode='group', category_orders={'weekday':weekdays_order})
# fig_maxpc.update_layout(
# )
# fig_maxpc.for_each_annotation(lambda x: x.update(text=x.text.split('=')[-1]))
# fig_maxpc.show()
fig_maxpc = go.Figure(data=go.Bar(
    x=max_price_change['weekday'], y=max_price_change['max change'], marker_color=max_price_change['color']
))
fig_maxpc.update_layout(
    title_text=f"Max Price Change for {trading_pair} in {month_year_filter}",
    height=wkday_fig_height,
    width=wkday_fig_width,
    xaxis=dict(categoryorder='array', categoryarray=weekdays_order),
)
fig_maxpc.show()

In [27]:
# fig_minpc = px.bar(min_price_change, x='weekday', y='min change', facet_row='month-year',
#                  barmode='group', category_orders={'weekday':weekdays_order})
# fig_minpc.update_layout(
# )
# fig_minpc.for_each_annotation(lambda x: x.update(text=x.text.split('=')[-1]))
# fig_minpc.show()
fig_minpc = go.Figure(data=go.Bar(
    x=min_price_change['weekday'], y=min_price_change['min change'], marker_color=min_price_change['color']
))
fig_minpc.update_layout(
    title_text=f"Min Price Change for {trading_pair} in {month_year_filter}",
    height=wkday_fig_height,
    width=wkday_fig_width,
    xaxis=dict(categoryorder='array', categoryarray=weekdays_order),
)
fig_minpc.show()

In [28]:
df_weekday_candles.head()

,candle_id,date,start,trading_pair,open,high,low,close,volume,price_change,price_direction,weekday,month,month-year,percent_change
0,ARB-1734480000,2024-12-17 16:00:00-08:00,1734480000,ARB-USD,0.9492,0.9558,0.8510,0.8615,9782590.86,-0.0877,Negative,Tuesday,12,2024-12,-9.239359
1,ARB-1734393600,2024-12-16 16:00:00-08:00,1734393600,ARB-USD,1.0034,1.0167,0.9343,0.9489,6079050.40,-0.0545,Negative,Monday,12,2024-12,-5.431533
2,ARB-1734307200,2024-12-15 16:00:00-08:00,1734307200,ARB-USD,1.0169,1.0546,0.9643,1.0032,11107753.24,-0.0137,Negative,Sunday,12,2024-12,-1.347232
3,ARB-1735516800,2024-12-29 16:00:00-08:00,1735516800,ARB-USD,0.7414,0.7702,0.7161,0.7319,14215223.65,-0.0095,Negative,Sunday,12,2024-12,-1.281360
4,ARB-1735430400,2024-12-28 16:00:00-08:00,1735430400,ARB-USD,0.7755,0.7819,0.7326,0.7411,2435685.80,-0.0344,Negative,Saturday,12,2024-12,-4.435848


In [55]:
df_weekday_prices = df_weekday_candles[['month-year', 'weekday', 'price_change']]
fig_wkday_prices = px.box(df_weekday_prices, x='month-year', y='price_change', facet_row='weekday',
                          category_orders={'month-year':month_years_order, 'weekday': weekdays_order})
fig_wkday_prices.for_each_annotation(lambda x: x.update(text=x.text.split('=')[-1]))
fig_wkday_prices.update_layout(
    height=wkday_fig_height+250,
    width=wkday_fig_width-450
)
fig_wkday_prices.show()

In [107]:
df_max_min = df_weekday_candles[['month-year', 'weekday', 'price_change']].groupby(['month-year', 'weekday'], as_index=False).agg(
    max=pd.NamedAgg(column='price_change', aggfunc='max'),
    min=pd.NamedAgg(column='price_change', aggfunc='min'),
    avg=pd.NamedAgg(column='price_change', aggfunc='mean'),
    )
df_max_min.head()

,month-year,weekday,max,min,avg
0,2024-08,Friday,0.0183,0.0000,0.005880
1,2024-08,Monday,0.0201,-0.0457,-0.010375
2,2024-08,Saturday,-0.0033,-0.0415,-0.023660
3,2024-08,Sunday,0.0367,-0.0830,-0.017225
4,2024-08,Thursday,0.0414,-0.0502,0.002560


In [110]:
fig_max_min = px.bar(df_max_min, x='month-year', y=['avg', 'max', 'min'], facet_row='weekday', barmode='group',
           category_orders={'weekday':weekdays_order, 'month_years':month_years_order})
fig_max_min.for_each_annotation(lambda x: x.update(text=x.text.split('=')[-1]))
fig_max_min.update_layout(
    height=wkday_fig_height+450,
    width=wkday_fig_width-250,
)
fig_max_min.show()